In [1]:
import os
import numpy as np
import re
import nltk
from sklearn.feature_extraction.text import CountVectorizer
from nltk.corpus import stopwords
from nltk import word_tokenize
import random
import math
import time
import operator

In [3]:
def cleaning(raw_data):
    letters_only = re.sub("[^a-zA-Z]", " ", raw_data)
    words = letters_only.lower().split()
    #words = word_tokenize(raw_data)
    stops = set(stopwords.words("english"))
    meaningful_words = [w for w in words if not w in stops]
    return meaningful_words

In [6]:
def load_data(dirc):
    # output a list of string
    package = os.listdir(dirc)
    string_list = []
    for name in package:
        file_dir = dirc + name
        string_list.append(cleaning(open(file_dir,encoding='latin-1').read()))
    return string_list

In [7]:
pos_dire = '/Users/kunqi/Downloads/review-polarity/tokens/pos/'
pos_data = load_data(pos_dire)
neg_dire = '/Users/kunqi/Downloads/review-polarity/tokens/neg/'
neg_data = load_data(neg_dire)
all_data = pos_data + neg_data

# take all the words appeared into account
def word_appeared(word_list):
    word_appear = []
    for samp in word_list:
        for w in samp:
            if w not in word_appear:
                word_appear.append(w) 
    return word_appear

w_appeared = word_appeared(all_data)


In [8]:
print(w_appeared[200:240])

['undoing', 'wins', 'consequences', 'dire', 'intensely', 'personal', 'filmmaker', 'interiors', 'hearts', 'minds', 'particular', 'trademarks', 'stripped', 'minimalist', 'style', 'flat', 'expressionless', 'dialogue', 'use', 'natural', 'sounds', 'music', 'background', 'twice', 'narration', 'segment', 'credits', 'heavy', 'drumbeat', 'accompanying', 'bagpipes', 'rest', 'scored', 'punctuate', 'thematic', 'elements', 'incessant', 'clanking', 'creaking', 'armor']


In [44]:
# order of feature vector will be total number of words appeared
def vectorize(to_vec,voc):
    vec = [0 for i in range(len(voc))]
    for word in to_vec:
        vec[voc.index(word)] += 1
    return vec

def normalize(vec):
    return (vec - np.mean(vec))/np.std(vec)

def get_set(data,label,voc):
    feature_set = []
    lable_set = [label for i in range(len(data))]
    for sam in data:
        vec = np.array(vectorize(sam,voc))
        nor = normalize(vec)
        feature_set.append(nor)
    formed_set = list(zip(feature_set,lable_set))           
    return formed_set

pos_set = get_set(pos_data,1,w_appeared)
neg_set = get_set(neg_data,-1,w_appeared)

pos_neg = pos_set + neg_set

#split data 
random.shuffle(pos_neg)
training_set = pos_neg[:1000]
test_set = pos_neg[1000:]
       

In [42]:
def eu_distance(v1,v2):
    return math.sqrt(sum((v1-v2)**2))

def KNN_predict(training_set,test_sample,K):
    dis_list = []
    random.shuffle(training_set)
    for point in training_set:
        dis_list.append((eu_distance(point[0],test_sample),point[1]))
    dis_list.sort(key=operator.itemgetter(0))
    top_K = dis_list[:K]
    pos = 0
    neg = 0
    for label in top_K:
        if label[1] == 1:
            pos += 1
        else:
            neg += 1
    if pos > neg:
        return 1
    else:
        return -1

def evaluate(training_set,test_set,K):
    test_results = [(KNN_predict(training_set,x,K),y) for (x,y) in test_set]
    corr = sum(int(x==y) for (x,y) in test_results)
    print("accuracy:%f"%float(corr/len(test_set)))    

print(KNN_predict(training_set,test_set[5][0],5),"res",test_set[5][1])

-1 res -1


In [45]:
start = time.time()
evaluate(training_set,test_set,5)
end = time.time()
print("elasp time",end-start)

accuracy:0.625000
elasp time 1336.894073009491
